In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.utils.vis_utils import plot_model
from keras.initializers import RandomNormal
from keras.regularizers import l2
import tensorflow as tf
import tensorflow.keras.optimizers
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from keras.layers import LeakyReLU
from keras.regularizers import l2
from keras.layers import Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from random import seed
from random import choice
from tqdm import tqdm
import ast

<h1>Function Library</h2>

In [ ]:
# This is the function used to build the two output structure of the models

def build_multi_output_model(input_size, stage_1_hidden_layers, stage_2_hidden_layers, 
                            stage_1_neurons, stage_2_neurons, dropout_rate, l2):
    input_size=input_size
    stage1_counter = 0
    stage2_counter = 0
    input_layer = keras.layers.Input(input_size, 
                                     name="input")
    while stage1_counter < (stage_1_hidden_layers):
        if stage1_counter == 0:
            x = keras.layers.Dense(stage_1_neurons, activation="relu", 
                                   kernel_regularizer=regularizers.l2(l2)
                                  )(input_layer)
            x = keras.layers.Dropout(dropout_rate)(x)
            stage1_counter += 1
        else:
            x = keras.layers.Dense(stage_1_neurons, activation="relu",
                                  kernel_regularizer=regularizers.l2(l2)
                                  )(x)
            x = keras.layers.Dropout(dropout_rate)(x)
            stage1_counter += 1    
    count_output = keras.layers.Dense(1, activation="sigmoid",
                                      name="count_output")(x)
    while stage2_counter < (stage_2_hidden_layers):
        if stage1_counter == 0:
            x = keras.layers.Dense(stage_2_neurons, activation=tf.keras.layers.LeakyReLU(alpha=0.1), 
                                   kernel_regularizer=regularizers.l2(l2)
                                  )(count_output)
            x = keras.layers.Dropout(dropout_rate)(x)
            stage2_counter += 1
        else:
            x = keras.layers.Dense(stage_2_neurons, activation=tf.keras.layers.LeakyReLU(alpha=0.1),
                                  kernel_regularizer=regularizers.l2(l2)
                                  )(x)
            x = keras.layers.Dropout(dropout_rate)(x)
            stage2_counter += 1
    class_output = keras.layers.Dense(4, activation="softmax",
                                      name="class_output")(x)
    model = keras.Model(inputs=input_layer, outputs=[count_output, class_output]) 
    return model


def compile_model(model, parameters, bin_weight, cla_weight):
    if optimal_parameters[-1] == 'adam':
        opt = 'adam'
    else:
        opt = tf.keras.optimizers.Adam(learning_rate = optimal_parameters[-1])
        
    model.compile(loss=["binary_crossentropy", "categorical_crossentropy"], 
              optimizer=opt, loss_weights=[bin_weight, cla_weight])


# Generate a random configuration of hyperparameters from the search space
def generate_configuration(first_stage_hl, second_stage_hl, first_stage_n, second_stage_n, 
                           dropout_rate, l2, learning_rate):
    c1 = choice(first_stage_hl)
    c2 = choice(second_stage_hl)
    c3 = choice(first_stage_n)
    c4 = choice(second_stage_n)
    c5 = choice(dropout_rate)
    c6 = choice(l2)
    c7 = choice(learning_rate)

    configuration = [19, c1, c2, c3, c4, c5, c6, c7] # 19 is the number of explanatory variables
    return configuration


# MC tuning process
def monte_carlo_tuning(x_train, y_train_1, y_train_2,
                       runs, n_samples):
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                          mode='min', verbose=0, patience=0, restore_best_weights=True)
    config_cache_loss = {}
    config_cache_count = {}
    config_cache_area = {}
    for _ in tqdm(range(runs)):
        configuration = generate_configuration(first_stage_hl, second_stage_hl, first_stage_n, second_stage_n, 
                           dropout_rate, l2, learning_rate)
        
        if all(configuration) not in config_cache_loss:
            sample_loss_cache = []
            for i in range(n_samples):
                model = build_multi_output_model(configuration[0], configuration[1],
                                               configuration[2], configuration[3], 
                                                configuration[4], configuration[5], configuration[6])
                
                if configuration[7] == 'adam':
                    opt = 'adam'
                else:
                    opt = tf.keras.optimizers.Adam(
                                    learning_rate = configuration[7])
                    
                model.compile(loss=["binary_crossentropy", "categorical_crossentropy"], 
                                                optimizer=opt,
                                                loss_weights=[0.5, 0.5])
                history = model.fit(x_train, [y_train_1, y_train_2], 
                    epochs=1, #update epochs
                    validation_split=0.2,
                    callbacks=EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=2),
                    verbose=0)   
                sample_loss = history.history['val_loss'][-1]
                sample_loss_cache.append(sample_loss)
            loss = sum(sample_loss_cache)/n_samples
            config_cache_loss[str(configuration)]  = loss
    return config_cache_loss


# Given a datalist and a number of breakpoints, this function finds the break point values such that the
# variance in each category is minimised. This code is inspired by: https://gist.github.com/drewda/1299198
# The code at the link provided does not work properly, however changes have been made here to fix it. 
def getJenksBreaks(dataList, numClass):
    dataList = list(dataList)
    dataList.sort()
    mat1 = []
    for i in range(0,len(dataList)+1):
        temp = []
        for j in range(0,numClass+1):
            temp.append(0)
        mat1.append(temp)
    mat2 = []
    for i in range(0,len(dataList)+1):
        temp = []
        for j in range(0,numClass+1):
            temp.append(0)
        mat2.append(temp)
    for i in range(1,numClass+1):
        mat1[1][i] = 1
        mat2[1][i] = 0
        for j in range(2,len(dataList)+1):
            mat2[j][i] = float('inf')
    v = 0.0
    for l in range(2,len(dataList)+1):
        s1 = 0.0
        s2 = 0.0
        w = 0.0
        for m in range(1,l+1):
            i3 = l - m + 1
            val = float(dataList[i3-1])
            s2 += val * val
            s1 += val
            w += 1
            v = s2 - (s1 * s1) / w
            i4 = i3 - 1
            if i4 != 0:
                for j in range(2,numClass+1):
                    if mat2[l][j] >= (v + mat2[i4][j - 1]):
                        mat1[l][j] = i3
                        mat2[l][j] = v + mat2[i4][j - 1]
        mat1[l][1] = 1
        mat2[l][1] = v
    k = len(dataList)
    breaks = []
    for i in range(0,numClass+1):
        breaks.append(min(dataList))
    countNum = numClass

    while countNum >= 2:#print "rank = " + str(mat1[k][countNum])
        id = int((mat1[k][countNum]) - 2)
        value = dataList[id]
        breaks.append(value)
        k = int((mat1[k][countNum] - 1))
        countNum -= 1
    breaks.append(max(dataList))
    breaks = list(set(breaks))
    breaks.sort()
    return(breaks)
    # The first output number is the smallest value in the input
    # Following output numbers are inclusive upper bounds


# The next two functions are used to determine the goodness of variance fit, determining how well the function above
# performed (in other words, it measures how well using a specific number of breakpoints applies to the data)
# This code is informed by a stack exchange comment availible at this link:
# https://stats.stackexchange.com/questions/143974/jenks-natural-breaks-in-python-how-to-find-the-optimum-number-of-breaks

def goodness_of_variance_fit(array, nclasses):
    
        classes = getJenksBreaks(array, nclasses)
        classified = np.array([classify(i, classes) for i in array])
        maxz = max(classified)
        zone_indices = [[idx for idx, val in enumerate(classified) if zone + 1 == val] for zone in range(maxz)]
        sdam = np.sum((array - array.mean()) ** 2)
        array_sort = [np.array([array[index] for index in zone]) for zone in zone_indices]
        sdcm = sum([np.sum((classified - classified.mean()) ** 2) for classified in array_sort])
        gvf = (sdam - sdcm) / sdam
        return gvf, classes
    
def classify(value, breaks):
    for i in range(1, len(breaks)):
        if value < breaks[i]:
            return i
    return len(breaks) - 1


# This function iterates over various numbers of breakpoints and uses the function above to find the best 
# number of break points

def optimal_classes(data, gvf_threshold, min_classes):

    gvf= 0.0
    nclasses = min_classes

    while gvf < gvf_threshold:
            gvf, classes = goodness_of_variance_fit(data, nclasses)
            if gvf < gvf_threshold:
                gvf, classes = goodness_of_variance_fit(data, nclasses)
                print('Classes Trialed: ', nclasses)
                print('gvf value :', gvf)
                print('Outcome: gvf value insufficient, further trials will be attempted')
                print()
                nclasses += 1
            else:
                print('Classes Trialed: ', nclasses)
                print('gvf value: ', gvf)
                print('Outcome: gvf value broke threshold, breaks found.')
                print('Breaks: ', classes)
    return classes
                

def plot_multiclass_roc(y_test, y_pred, n_classes, figsize=(17, 6)):

    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    y_test_dummies = pd.get_dummies(y_test, drop_first=False).values
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_dummies[:, i], y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    fig, ax = plt.subplots(figsize=figsize)
    ax.plot([0, 1], [0, 1], 'k--')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title('ROC Curves for Each Class')
    for i in range(n_classes):
        ax.plot(fpr[i], tpr[i], label='ROC curve (area = %0.6f) for label %i' % (roc_auc[i], i))
    ax.legend(loc="best")
    ax.grid(alpha=.4)
#     plt.savefig("class_model_ROC.pdf") <- Uncomment to save graph
    plt.show()
    
    return roc_auc
    
# Test different different weight combinations 
def loss_weight_ratio_testing(input_model, ratio_increase):
    weight_1 = 0
    weight_2 = 10
    count_roc =[]
    class_ovr_w_roc =[]
    model_histories = []
    weight_ratio = []
    
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)
    
    while weight_1 <= 10:
        model = input_model
        
        model.compile(loss=["binary_crossentropy", "categorical_crossentropy"], 
              optimizer='adam',
              loss_weights=[weight_1, weight_2])

        history = model.fit(x_train, [y_train_count, y_train_area], 
                    epochs=15,#30
                    verbose = 0,
                    validation_split=0.2
                   )
        
        y_pred = model.predict(x_test)
        count_predictions, area_predictions = y_pred[0], y_pred[1]
        
        c_auc = roc_auc_score(y_test_count, count_predictions)
        
        w_ovr = roc_auc_score(y_test_area, area_predictions, multi_class="ovo",
                                     average="weighted")

        # summarize scores
        print('Count Weight: ', weight_1, 'Class Weight: ', weight_2)
        print('Count ROC AUC=%.6f' % (c_auc))
        print('Class Weighted OVR ROC:', w_ovr)
        print()                  
        count_roc.append(c_auc)
        class_ovr_w_roc.append(w_ovr)
        model_histories.append(history)
        wr = 0
        if weight_2 == 0:
            weight_ratio.append(wr)
        else:
            wr = (weight_1/weight_2)
            weight_ratio.append(wr)
        weight_1 += ratio_increase
        weight_2 = 10-weight_1
    return count_roc, class_ovr_w_roc

<h1>Load & Preprocess Data</h1>

In [ ]:
data = pd.read_csv("landslide_data.csv") 

In [ ]:
# First step is to use Fischer-Jenks to get the area classes

# Create a df of non-zero Area_Slide
filtered_data = data[data['Area_Slide'] > 0]
non_zero_slide_area = filtered_data['Area_Slide']
non_zero_slide_area = np.array(non_zero_slide_area)

# Find the optimal breaks in non-zero Area_Slide, gvf threshold of 0.85, start by trying two classes
optimal_classes = optimal_classes(non_zero_slide_area, 0.85, 2)

In [ ]:
# Apply a class value to each record based on the above results
data.loc[(data["Area_Slide"] == 0) & (data["Area_Slide"] < optimal_classes[0]), "Slide_Class"] = "Class 0" 
data.loc[(data["Area_Slide"] >= optimal_classes[0]) & (data["Area_Slide"] <= optimal_classes[1]), "Slide_Class"] = "Class 1" 
data.loc[(data["Area_Slide"] > optimal_classes[1]) & (data["Area_Slide"] <= optimal_classes[2]), "Slide_Class"] = "Class 2" 
data.loc[(data["Area_Slide"] > optimal_classes[2]) & (data["Area_Slide"] <= optimal_classes[3]), "Slide_Class"] = "Class 3" 
data.loc[(data["Area_Slide"] > optimal_classes[3]) & (data["Area_Slide"] <= optimal_classes[4]), "Slide_Class"] = "Class 4"
data.loc[(data["Area_Slide"] > optimal_classes[4]) & (data["Area_Slide"] <= optimal_classes[5]), "Slide_Class"] = "Class 5"
data["Slide_Class"].value_counts()

# Observe: 1) The imbalanced distribution, and 2) Class 5 only has one record

In [ ]:
# Merge select classes, rationale in paper
data.loc[(data["Area_Slide"] == 0) & (data["Area_Slide"] < optimal_classes[0]), "Slide_Class"] = "Class 0" 
data.loc[(data["Area_Slide"] >= optimal_classes[0]) & (data["Area_Slide"] <= optimal_classes[2]), "Slide_Class"] = "Class 1"
data.loc[(data["Area_Slide"] > optimal_classes[2]) & (data["Area_Slide"] <= optimal_classes[3]), "Slide_Class"] = "Class 2" 
data.loc[(data["Area_Slide"] > optimal_classes[3]) & (data["Area_Slide"] <= optimal_classes[5]), "Slide_Class"] = "Class 3"
data["Slide_Class"].value_counts()

In [ ]:
### Uncomment this cell to bypass the process of the three preceeding cells ###
data.loc[(data["Area_Slide"] == 0) & (data["Area_Slide"] < 0.054691), "Slide_Class"] = "Class 0" 
data.loc[(data["Area_Slide"] >= 0.054691) & (data["Area_Slide"] <= 2981.98645), "Slide_Class"] = "Class 1" 
data.loc[(data["Area_Slide"] > 2981.98645) & (data["Area_Slide"] <= 10821.34832), "Slide_Class"] = "Class 1" 
data.loc[(data["Area_Slide"] > 10821.34832) & (data["Area_Slide"] <= 29509.81466), "Slide_Class"] = "Class 2" 
data.loc[(data["Area_Slide"] > 29509.81466) & (data["Area_Slide"] <= 82850.15392), "Slide_Class"] = "Class 3"
data.loc[(data["Area_Slide"] > 82850.15392) & (data["Area_Slide"] <= 182966.8526), "Slide_Class"] = "Class 3"
data["Slide_Class"].value_counts()

In [ ]:
# Define dependant and explanatory variables
data_y = data['Slide_Class']
data_x = data.drop(['SU_ID', 'Count', 'Area_Slide', 'Slide_Class'], axis=1)
data_x = data_x.astype(float)


# Train/Test split
x_train, x_test, y_train_area, y_test_area = train_test_split(data_x, data_y, test_size=0.2, random_state=0)


# Standardize the explanatory variables
scaler = preprocessing.StandardScaler()

train_x_values = x_train.values
x_scaled1 = scaler.fit_transform(train_x_values)
x_train = pd.DataFrame(x_scaled1)

test_x_values = x_test.values
x_scaled2 = scaler.fit_transform(test_x_values)
x_test = pd.DataFrame(x_scaled2)


# Use SMOTE package to oversample training data
oversample = SMOTE()
x_train, y_train_area = oversample.fit_resample(x_train, y_train_area)


# Use the Slide_Class field to create a binary variable that 
# indicates whether or not there is a landslide in any capacity
y_train_count = []
y_test_count = []
for i in y_train_area:
    if i == "Class 0":
        y_train_count.append(0)
    else:
        y_train_count.append(1)
        
for i in y_test_area:
    if i == "Class 0":
        y_test_count.append(0)
    else:
        y_test_count.append(1)     
y_train_count = np.array(y_train_count)
y_test_count = np.array(y_test_count)


# encode class values as integers
encoder = LabelEncoder()

# encode training
encoder.fit(y_train_area)
encoded_Y = encoder.transform(y_train_area)
y_train_area = np_utils.to_categorical(encoded_Y)

# encode testing
# encoder.fit(y_test_area)
# encoded_Y_test = encoder.transform(y_test_area)
# y_test_area = np_utils.to_categorical(encoded_Y_test)

<h1>Modelling</h1>

In [ ]:
# Define the search space for MC tuning
first_stage_hl  = [6, 12, 18, 24, 48]
second_stage_hl = [6, 12, 18, 24]
first_stage_n   = [4, 8, 16, 32, 64]
second_stage_n  = [4, 8, 16, 32, 64]
dropout_rate    = [0.2, 0.4, 0.6, 0.8]
l2              = [1e-3, 1e-4, 1e-5]
learning_rate   = [1e-2, 1e-3, 1e-4, "adam"]

In [ ]:
# USE MONTE CARLO TUNING FUNCTION 

# Uncomment the code below to run the montecarlo tuner. Be advised, this process is time consuming and you may 
# prefer to simply reuse the values in the report. 

# config_cache = monte_carlo_tuning(x_train, y_train_count, y_train_area, runs=1000, n_samples=5)
# optimal_parameters_string = min(config_cache, key=config_cache.get)
# optimal_parameters = ast.literal_eval(optimal_parameters_string)

optimal_parameters = [19, 6, 12, 32, 4, 0.2, 1e-4, 'adam']

In [ ]:
lwr_model = build_multi_output_model(*optimal_parameters[:-1]) # Leave out the learning rate for now

In [ ]:
# This is also a time consuming process, uncomment the vaules in the next cell to reuse the results from the report
co, cl = loss_weight_ratio_testing(lwr_model, 1)

In [ ]:
iteration = list(range(1,12))
# co = [0.4697642202160677,
#   0.8732161398821352,
#   0.8738895262157222,
#   0.872348897639263,
#   0.8735841626784037,
#   0.8747538702127408,
#   0.8755926742685977,
#   0.8767841662705956,
#   0.8773556105481453,
#   0.8789159417536946,
#   0.8822436314449064]
# cl = [0.8670034294002861,
#   0.8719836649189152,
#   0.8721181496411214,
#   0.8706711380269829,
#   0.8717760611632063,
#   0.8731124107537576,
#   0.8740722810400376,
#   0.8746425163405755,
#   0.8752726948902124,
#   0.8747853344452116,
#   0.5008439865358839]


plt.figure(figsize=(6, 6))

plt.plot(iteration, cl, 'o-', color='blue')
plt.plot(iteration, co, 'o-', color='red')

plt.title('Loss Weight Ratio Testing ROC AUC Results')
plt.xlabel('Iteration')
plt.ylabel('ROC AUC Value (One Vs. Rest Weighted for Class Output)')
    
red_patch = mpatches.Patch(color='red', label='Binary Output')
blu_patch = mpatches.Patch(color='blue', label='Class Output')
plt.legend(handles=[red_patch, blu_patch])

plt.show()

plt.figure(figsize=(6, 6))

plt.plot(iteration, cl, 'o-', color='blue') 
plt.plot(iteration, co, 'o-', color='red') 

plt.ylim(0.865, 0.905) # adjust to fit as needed

plt.title('Loss Weight Ratio Testing ROC AUC Results Zoomed')
plt.xlabel('Iteration')
plt.ylabel('ROC AUC Value (One Vs. Rest Weighted for Class Output)')
    
red_patch = mpatches.Patch(color='red', label='Binary Output')
blu_patch = mpatches.Patch(color='blue', label='Class Output')
plt.legend(handles=[red_patch, blu_patch])
    
plt.show()

In [ ]:
baseline = LogisticRegression(solver='liblinear', random_state=0)
baseline.fit(x_train, y_train_count)
baseline_pred = baseline.predict_proba(x_test)
baseline_pred = baseline_pred[:,1]

In [ ]:
# the random initial weightings of the following models mean that if 0.5 ROC AUC results are produced,
# you should re-run them and they will work properly. 

<h2>Binary Model</h2>

In [ ]:
# Define model and checkpoints
binary_model = build_multi_output_model(*optimal_parameters[:-1])

compile_model(binary_model, optimal_parameters, 10, 0)

binary_checkpoint = ModelCheckpoint("Binary_Checkpoint", monitor='val_loss', verbose=1,
    save_best_only=True, mode='min', save_freq='epoch')

# Train model
binary_model_history = binary_model.fit(x_train, [y_train_count, y_train_area], 
                    epochs=30,
                    callbacks=[binary_checkpoint],
                    verbose = 1,
                    validation_split=0.2
                   )

In [ ]:
# load model
best_binary_model = tensorflow.keras.models.load_model('Binary_Checkpoint')

# Make predictions
binary_y_pred=best_binary_model.predict(x_test)
binary_count_predictions, binary_area_predictions = binary_y_pred[0], binary_y_pred[1]

binary_auc = roc_auc_score(y_test_count, binary_count_predictions)
baseline_auc = roc_auc_score(y_test_count, baseline_pred)
print('Baseline:     ROC AUC=%.6f' % (baseline_auc))
print('Binary Model: ROC AUC=%.6f' % (binary_auc))

plt.figure(figsize=(6, 6))
ns_probs = [0 for _ in range(len(y_test_count))]
ns1, ns2, _ = roc_curve(y_test_count, ns_probs)
nn1, nn2, _ = roc_curve(y_test_count, binary_count_predictions)
bl1, bl2, _ = roc_curve(y_test_count, baseline_pred)
plt.plot(ns1, ns2, linestyle='--', label='No Skill')
plt.plot(nn1, nn2, marker='.', color='blue',label='ROC curve for Binary Model ('+'%.5f' % (binary_auc)+')')
plt.plot(bl1, bl2, marker='.', color='red', label='ROC curve for Baseline Model ('+'%.5f' % (baseline_auc)+')')

# axis labels
plt.title('ROC Curves for Binary Model and Baseline')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# show the legend
plt.legend()
# show the plot
plt.plot(figsize=(6, 12))
plt.show()

best_epochs_b = np.argmin(binary_model_history.history['val_loss'])

history = binary_model_history
plt.subplot()
plt.title('Binary Model Overall loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val')
plt.axvline(x=best_epochs_b, label='Epoch with lowest val_loss = ' + '%.0f' % (best_epochs_b), color='green')
plt.legend()
# plt.savefig("binary_model_loss.pdf")

<h2>Class Model</h2>

In [ ]:
# Define model and checkpoints
class_model  = build_multi_output_model(*optimal_parameters[:-1])

compile_model(class_model, optimal_parameters, 8, 2)

class_checkpoint = ModelCheckpoint("Class_Checkpoint", monitor='val_loss', verbose=1,
    save_best_only=True, mode='min', save_freq='epoch')

# Train model
class_model_history = class_model.fit(x_train, [y_train_count, y_train_area], 
                    epochs=30,
                    callbacks=[class_checkpoint],
                    verbose = 1,
                    validation_split=0.2
                   )

In [ ]:
# load best model
best_class_model = tensorflow.keras.models.load_model('Class_Checkpoint')

# Make predictions
class_y_pred=best_class_model.predict(x_test)
class_count_predictions, class_area_predictions = class_y_pred[0], class_y_pred[1]

# Test Predictions
macro_roc_auc_ovo = roc_auc_score(y_test_area, class_area_predictions, multi_class="ovo",
                                  average="macro")
weighted_roc_auc_ovo = roc_auc_score(y_test_area, class_area_predictions, multi_class="ovo",
                                     average="weighted")
macro_roc_auc_ovr = roc_auc_score(y_test_area, class_area_predictions, multi_class="ovr",
                                  average="macro")
weighted_roc_auc_ovr = roc_auc_score(y_test_area, class_area_predictions, multi_class="ovr",
                                     average="weighted")
print("One-vs-One ROC AUC scores:\n{:.6f} (macro),\n{:.6f} "
      "(weighted by prevalence)"
      .format(macro_roc_auc_ovo, weighted_roc_auc_ovo))
print("One-vs-Rest ROC AUC scores:\n{:.6f} (macro),\n{:.6f} "
      "(weighted by prevalence)"
      .format(macro_roc_auc_ovr, weighted_roc_auc_ovr))

multiclass_roc_auc = plot_multiclass_roc(y_test_area, class_area_predictions, n_classes=4, figsize=(6, 6))

best_epoch = np.argmin(class_model_history.history['val_loss'])

history = class_model_history
plt.subplot()
plt.title('Class Model Overall loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val')
plt.axvline(x=best_epoch, label='Epoch with lowest val_loss = ' + '%.0f' % (best_epoch), color='green')
plt.legend()
# plt.savefig("class_model_loss.pdf")

<h1>Ablation Study</h1>

In [ ]:
def remove_variable(data, variable1, variable2):

    # Repeat preprocessing, but do so with one variable removed
    data_y = data['Slide_Class']
    data_x = data.drop(['SU_ID', 'Count', 'Area_Slide', 'Slide_Class'], axis=1)
    if variable1 == variable2:
        data_x = data_x.drop(variable1, axis=1)
        data_x = data_x.astype(float)
    else:
        data_x = data_x.drop(variable1, axis=1)
        data_x = data_x.drop(variable2, axis=1)
        data_x = data_x.astype(float)

    
    x_train, x_test, y_train_area, y_test_area = train_test_split(data_x, 
                                                                  data_y, 
                                                                  test_size=0.2, 
                                                                  random_state=0)
    
    scaler = preprocessing.StandardScaler()

    train_x_values = x_train.values
    x_scaled1 = scaler.fit_transform(train_x_values)
    x_train = pd.DataFrame(x_scaled1)

    test_x_values = x_test.values
    x_scaled2 = scaler.fit_transform(test_x_values)
    x_test = pd.DataFrame(x_scaled2)
    
    oversample = SMOTE()
    x_train, y_train_area = oversample.fit_resample(x_train, y_train_area)
    
    encoder = LabelEncoder()

    #training
    encoder.fit(y_train_area)
    encoded_Y = encoder.transform(y_train_area)
    y_train = np_utils.to_categorical(encoded_Y)

    #testing
    encoder.fit(y_test_area)
    encoded_Y_test = encoder.transform(y_test_area)
    y_test = np_utils.to_categorical(encoded_Y_test)
    
    y_train_count = []
    y_test_count = []

    for i in y_train_area:
        if i == "Class 0":
            y_train_count.append(0)
        else:
            y_train_count.append(1)

    for i in y_test_area:
        if i == "Class 0":
            y_test_count.append(0)
        else:
            y_test_count.append(1)

    y_train_count = np.array(y_train_count)
    y_test_count = np.array(y_test_count)
    
    return x_train, x_test, y_train, y_test_area, y_train_count, y_test_count

In [ ]:
explanatory_variable_types = ['Area_SU', 'DIst', 'Slope', 'VRM', 'PRC', 'PLC', 'EAST', 'North', 'PGA', 'Relief']
remove_com = []

for i in explanatory_variable_types:
    temp = []
    for j in data.columns:
        if i in j:
            temp.append(j)
    if temp != []:
        if len(temp) == 1:
            temp.append(temp[0])
            remove_com.append(temp)
        else:
            remove_com.append(temp)

In [ ]:
ablation_results = []
for j in tqdm(remove_com):
    auc0 = []
    auc1 = []
    auc2 = []
    auc3 = []
    for itr in range(10):
        ab_x_train, ab_x_test, ab_y_train, ab_y_test_area, ab_y_train_count, ab_y_test_count = remove_variable(data, *j)
        
        ab_class_model = build_multi_output_model(ab_x_train.shape[1],*optimal_parameters[1:-1])
        
        compile_model(ab_class_model, optimal_parameters, 8, 2)

        ab_class_history = ab_class_model.fit(ab_x_train, [ab_y_train_count, ab_y_train], 
                            epochs=30,
                            verbose = 0,
                            validation_split=0.2
                           )
        class_y_pred=ab_class_model.predict(ab_x_test)
        class_area_predictions = class_y_pred[1]
        
        # Compute ROC curve and ROC area for each class
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        
                        
        y_test_dummies = pd.get_dummies(ab_y_test_area, drop_first=False).values
        for i in range(4):
            fpr[i], tpr[i], _ = roc_curve(y_test_dummies[:, i], class_area_predictions[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])
            if i == 0:
                auc0.append(roc_auc[i])
            if i == 1:
                auc1.append(roc_auc[i])
            if i == 2:
                auc2.append(roc_auc[i])
            if i == 3:
                auc3.append(roc_auc[i])
    result = [j, auc0, auc1, auc2, auc3]
    ablation_results.append(result)


In [ ]:
# Rename variables for more clear understanding of the following cell
area    = ablation_results[0][1:]
dist    = ablation_results[1][1:]
slope   = ablation_results[2][1:]
vrm     = ablation_results[3][1:]
prc     = ablation_results[4][1:]
plc     = ablation_results[5][1:]
east    = ablation_results[6][1:]
north   = ablation_results[7][1:]
pga     = ablation_results[8][1:]
relief  = ablation_results[9][1:]

In [ ]:
fig, ax = plt.subplots(figsize=(18, 10))

colors = ['green', 'orange', 'brown', 'red']
green = dict(color=colors[0])
orange = dict(color=colors[1])
brown = dict(color=colors[2])
red = dict(color=colors[3])

ax.boxplot(area[0], positions=[1], boxprops=green, medianprops=green, whiskerprops=green, capprops=green)
ax.boxplot(dist[0], positions=[2], boxprops=green, medianprops=green, whiskerprops=green, capprops=green)
ax.boxplot(slope[0], positions=[3], boxprops=green, medianprops=green, whiskerprops=green, capprops=green)
ax.boxplot(vrm[0], positions=[4], boxprops=green, medianprops=green, whiskerprops=green, capprops=green)
ax.boxplot(prc[0], positions=[5], boxprops=green, medianprops=green, whiskerprops=green, capprops=green)
ax.boxplot(plc[0], positions=[6], boxprops=green, medianprops=green, whiskerprops=green, capprops=green)
ax.boxplot(east[0], positions=[7], boxprops=green, medianprops=green, whiskerprops=green, capprops=green)
ax.boxplot(north[0], positions=[8], boxprops=green, medianprops=green, whiskerprops=green, capprops=green)
ax.boxplot(pga[0], positions=[9], boxprops=green, medianprops=green, whiskerprops=green, capprops=green)
ax.boxplot(relief[0], positions=[10], boxprops=green, medianprops=green, whiskerprops=green, capprops=green)

ax.boxplot(area[1], positions=[11], boxprops=orange, medianprops=orange, whiskerprops=orange, capprops=orange)
ax.boxplot(dist[1], positions=[12], boxprops=orange, medianprops=orange, whiskerprops=orange, capprops=orange)
ax.boxplot(slope[1], positions=[13], boxprops=orange, medianprops=orange, whiskerprops=orange, capprops=orange)
ax.boxplot(vrm[1], positions=[14], boxprops=orange, medianprops=orange, whiskerprops=orange, capprops=orange)
ax.boxplot(prc[1], positions=[15], boxprops=orange, medianprops=orange, whiskerprops=orange, capprops=orange)
ax.boxplot(plc[1], positions=[16], boxprops=orange, medianprops=orange, whiskerprops=orange, capprops=orange)
ax.boxplot(east[1], positions=[17], boxprops=orange, medianprops=orange, whiskerprops=orange, capprops=orange)
ax.boxplot(north[1], positions=[18], boxprops=orange, medianprops=orange, whiskerprops=orange, capprops=orange)
ax.boxplot(pga[1], positions=[19], boxprops=orange, medianprops=orange, whiskerprops=orange, capprops=orange)
ax.boxplot(relief[1], positions=[20], boxprops=orange, medianprops=orange, whiskerprops=orange, capprops=orange)

ax.boxplot(area[2], positions=[21], boxprops=brown, medianprops=brown, whiskerprops=brown, capprops=brown)
ax.boxplot(dist[2], positions=[22], boxprops=brown, medianprops=brown, whiskerprops=brown, capprops=brown)
ax.boxplot(slope[2], positions=[23], boxprops=brown, medianprops=brown, whiskerprops=brown, capprops=brown)
ax.boxplot(vrm[2], positions=[24], boxprops=brown, medianprops=brown, whiskerprops=brown, capprops=brown)
ax.boxplot(prc[2], positions=[25], boxprops=brown, medianprops=brown, whiskerprops=brown, capprops=brown)
ax.boxplot(plc[2], positions=[26], boxprops=brown, medianprops=brown, whiskerprops=brown, capprops=brown)
ax.boxplot(east[2], positions=[27], boxprops=brown, medianprops=brown, whiskerprops=brown, capprops=brown)
ax.boxplot(north[2], positions=[28], boxprops=brown, medianprops=brown, whiskerprops=brown, capprops=brown)
ax.boxplot(pga[2], positions=[29], boxprops=brown, medianprops=brown, whiskerprops=brown, capprops=brown)
ax.boxplot(relief[2], positions=[30], boxprops=brown, medianprops=brown, whiskerprops=brown, capprops=brown)

ax.boxplot(area[3], positions=[31], boxprops=red, medianprops=red, whiskerprops=red, capprops=red)
ax.boxplot(dist[3], positions=[32], boxprops=red, medianprops=red, whiskerprops=red, capprops=red)
ax.boxplot(slope[3], positions=[33], boxprops=red, medianprops=red, whiskerprops=red, capprops=red)
ax.boxplot(vrm[3], positions=[34], boxprops=red, medianprops=red, whiskerprops=red, capprops=red)
ax.boxplot(prc[3], positions=[35], boxprops=red, medianprops=red, whiskerprops=red, capprops=red)
ax.boxplot(plc[3], positions=[36], boxprops=red, medianprops=red, whiskerprops=red, capprops=red)
ax.boxplot(east[3], positions=[37], boxprops=red, medianprops=red, whiskerprops=red, capprops=red)
ax.boxplot(north[3], positions=[38], boxprops=red, medianprops=red, whiskerprops=red, capprops=red)
ax.boxplot(pga[3], positions=[39], boxprops=red, medianprops=red, whiskerprops=red, capprops=red)
ax.boxplot(relief[3], positions=[40], boxprops=red, medianprops=red, whiskerprops=red, capprops=red)

ax.axhline(y=multiclass_roc_auc[0], xmin=0, xmax=0.25, color='green')
ax.axhline(y=multiclass_roc_auc[1], xmin=0.25, xmax=0.5, color='orange')
ax.axhline(y=multiclass_roc_auc[2], xmin=0.5, xmax=0.75, color='brown')
ax.axhline(y=multiclass_roc_auc[3], xmin=0.75, xmax=1, color='red')

green_patch = mpatches.Patch(color='green', label='Class 0 Ablation Results With Class Model ROC AUC Reference')
orange_patch = mpatches.Patch(color='orange', label='Class 1 Ablation Results With Class Model ROC AUC Reference')
brown_patch = mpatches.Patch(color='brown', label='Class 2 Ablation Results With Class Model ROC AUC Reference')
red_patch = mpatches.Patch(color='red', label='Class 3 Ablation Results With Class Model ROC AUC Reference')

ax.legend(handles=[green_patch, orange_patch, brown_patch, red_patch])

plt.xticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40], 
           ['Area_SU', 'Dist2Str', 'Slope', 'VRM', 'PRC', 'PLC', 'Eastness', 'Northness', 'PGA', 'Relief', 'Area_SU', 'Dist2Str', 'Slope', 'VRM', 'PRC', 'PLC', 'Eastness', 'Northness', 'PGA', 'Relief', 'Area_SU', 'Dist2Str', 'Slope', 'VRM', 'PRC', 'PLC', 'Eastness', 'Northness', 'PGA', 'Relief', 'Area_SU', 'Dist2Str', 'Slope', 'VRM', 'PRC', 'PLC', 'Eastness', 'Northness', 'PGA', 'Relief'],
          rotation='vertical')

plt.title('Ablation Study Results, Class Breakdown')
plt.xlabel('Variable')
plt.ylabel('ROC AUC')
plt.savefig("class_ablation.pdf")
plt.show()

In [ ]:
bin_ablation_results = []

for i in tqdm(remove_com):
    vals = []
    for itr in range(10):
        ab_x_train, ab_x_test, ab_y_train, ab_y_test_area, ab_y_train_count, ab_y_test_count = remove_variable(data, *i)

        ab_binary_model = build_multi_output_model(ab_x_train.shape[1],*optimal_parameters[1:-1])
        
        compile_model(ab_binary_model, optimal_parameters, 10, 0)
        
        bin_history = ab_binary_model.fit(ab_x_train, [ab_y_train_count, ab_y_train], 
                            epochs=30,
                            verbose = 0,
                            validation_split=0.2
                           )
        binary_y_pred=ab_binary_model.predict(ab_x_test)
        binary_predictions = binary_y_pred[0]
        binary_auc = roc_auc_score(ab_y_test_count, binary_predictions)   
        vals.append(binary_auc)
        
    result = [i, vals]
    bin_ablation_results.append(result)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

colors = ['green', 'orange', 'brown', 'red']
green = dict(color=colors[0])
orange = dict(color=colors[1])
brown = dict(color=colors[2])
red = dict(color=colors[3])


plt.boxplot(bin_ablation_results[0][1], positions=[1])
plt.boxplot(bin_ablation_results[1][1], positions=[2])
plt.boxplot(bin_ablation_results[2][1], positions=[3])
plt.boxplot(bin_ablation_results[3][1], positions=[4])
plt.boxplot(bin_ablation_results[4][1], positions=[5])
plt.boxplot(bin_ablation_results[5][1], positions=[6])
plt.boxplot(bin_ablation_results[6][1], positions=[7])
plt.boxplot(bin_ablation_results[7][1], positions=[8])
plt.boxplot(bin_ablation_results[8][1], positions=[9])
plt.boxplot(bin_ablation_results[9][1], positions=[10])

plt.axhline(y=binary_auc, color='green', label='OVR Weighted ROC AUC Average of Binary Model')
plt.legend()

plt.xticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 
           ['Area_SU', 'Dist2Str', 'Slope', 'VRM', 'PRC', 'PLC', 'Eastness', 'Northness', 'PGA', 'Relief'])

plt.title('Ablation Study Results, Binary Model')
plt.xlabel('Variable')
plt.ylabel('ROC AUC')
# plt.savefig("binary_ablation.pdf")
plt.show()